<a href="https://colab.research.google.com/github/arsendoinychko/Cp-decomposition/blob/main/Tucker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import time

# Hyperparameters
batch_size = 64
learning_rate = 0.001
num_epochs = 2

# Loading and normalizing CIFAR-10 dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Loading the pre-trained VGG-16 and modifying it for 10 classes of CIFAR-10
class ModifiedVGG16(nn.Module):
    def __init__(self):
        super(ModifiedVGG16, self).__init__()
        self.vgg16 = torchvision.models.vgg16(pretrained=True)
        self.vgg16.classifier[6] = nn.Linear(4096, 10)  # Змінюємо останній повнозв'язний шар на 10 виходів

    def forward(self, x):
        x = self.vgg16(x)
        return x

# Initializing the model, loss function, and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ModifiedVGG16()

# Using multiple GPUs if available
if torch.cuda.device_count() > 1:
    print(f'Using {torch.cuda.device_count()} GPUs!')
    model = nn.DataParallel(model)

model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training the model
for epoch in range(num_epochs):
    start_time = time.time()
    running_loss = 0.0
    model.train()
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {running_loss / 100:.4f}')
            running_loss = 0.0
    epoch_duration = time.time() - start_time
    print(f'Epoch [{epoch + 1}/{num_epochs}] completed in {epoch_duration:.2f} seconds')

print('Finished Training')

# Testing the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

# Saving the model
torch.save(model.state_dict(), 'vgg16_cifar10_model.pth')

In [ ]:
# Testing the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

# Saving the model
torch.save(model.state_dict(), 'vgg16_cifar10_model.pth')

In [ ]:
print(model)

In [ ]:
if isinstance(model, nn.DataParallel):
    model = model.module

conv_layer1 = model.vgg16.features[0]
conv_layer2 = model.vgg16.features[2]
conv_layer3 = model.vgg16.features[5]
conv_layer4 = model.vgg16.features[7]
conv_layer5 = model.vgg16.features[10]
conv_layer6 = model.vgg16.features[12]
conv_layer7 = model.vgg16.features[14]
conv_layer8 = model.vgg16.features[17]
conv_layer9 = model.vgg16.features[19]
conv_layer10 = model.vgg16.features[21]
conv_layer11 = model.vgg16.features[24]
conv_layer12 = model.vgg16.features[26]
conv_layer13 = model.vgg16.features[28]

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import tensorly as tl
from tensorly.decomposition import tucker, partial_tucker

def tucker(layer, rank):
    W = layer.weight.data

    core, [last, first] = partial_tucker(W, modes=[0,1], ranks=rank, init='svd')

    first_layer = nn.Conv2d(in_channels=first.shape[0],
                                       out_channels=first.shape[1],
                                       kernel_size=1,
                                       padding=0,
                                       bias=False)

    core_layer = nn.Conv2d(in_channels=core.shape[1],
                                       out_channels=core.shape[0],
                                       kernel_size=layer.kernel_size,
                                       stride=layer.stride,
                                       padding=layer.padding,
                                       dilation=layer.dilation,
                                       bias=False)

    last_layer = nn.Conv2d(in_channels=last.shape[1],
                                       out_channels=last.shape[0],
                                       kernel_size=1,
                                       padding=0,
                                       bias=True)

    if layer.bias is not None:
        last_layer.bias.data = layer.bias.data

    fk = first.t_().unsqueeze_(-1).unsqueeze_(-1)
    lk = last.unsqueeze_(-1).unsqueeze_(-1)

    first_layer.weight.data = fk
    last_layer.weight.data = lk
    core_layer.weight.data = core

    new_layers = [first_layer, core_layer, last_layer]
    return new_layers

In [ ]:
new_layers1 = tucker(conv_layer1.cpu())
new_layers2 = tucker(conv_layer2.cpu())
new_layers3 = tucker(conv_layer3.cpu())
new_layers4 = tucker(conv_layer4.cpu())
new_layers5 = tucker(conv_layer5.cpu())
new_layers6 = tucker(conv_layer6.cpu())
new_layers7 = tucker(conv_layer7.cpu())
new_layers8 = tucker(conv_layer8.cpu())
new_layers9 = tucker(conv_layer9.cpu())
new_layers10 = tucker(conv_layer10.cpu())
new_layers11 = tucker(conv_layer11.cpu())
new_layers12 = tucker(conv_layer12.cpu())
new_layers13 = tucker(conv_layer13.cpu())

In [ ]:
model.vgg16.features[0] = nn.Sequential(*new_layer1).cuda()
model.vgg16.features[2] = nn.Sequential(*new_layer2).cuda()
model.vgg16.features[5] = nn.Sequential(*new_layer3).cuda()
model.vgg16.features[7] = nn.Sequential(*new_layer4).cuda()
model.vgg16.features[10] = nn.Sequential(*new_layer5).cuda()
model.vgg16.features[12] = nn.Sequential(*new_layer6).cuda()
model.vgg16.features[14] = nn.Sequential(*new_layer7).cuda()
model.vgg16.features[17] = nn.Sequential(*new_layer8).cuda()
model.vgg16.features[19] = nn.Sequential(*new_layer9).cuda()
model.vgg16.features[21] = nn.Sequential(*new_layer10).cuda()
model.vgg16.features[24] = nn.Sequential(*new_layer11).cuda()
model.vgg16.features[26] = nn.Sequential(*new_layer12).cuda()
model.vgg16.features[28] = nn.Sequential(*new_layer13).cuda()

In [ ]:
for epoch in range(num_epochs):
    start_time = time.time()
    running_loss = 0.0
    model.train()
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {running_loss / 100:.4f}')
            running_loss = 0.0
    epoch_duration = time.time() - start_time
    print(f'Epoch [{epoch + 1}/{num_epochs}] completed in {epoch_duration:.2f} seconds')

print('Finished Training')

In [ ]:
# Testing the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

# Saving the model
torch.save(model.state_dict(), 'vgg16_cifar10_model.pth')